# Day 5 : Hydrothermal Venture

## Part 1 : Puzzle Description

You come across a field of [hydrothermal vents](https://en.wikipedia.org/wiki/Hydrothermal_vent) on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```text
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like `1,1 -> 1,3` covers points `1,1`, `1,2`, and `1,3`.
- An entry like `9,7 -> 7,7` covers points `9,7`, `8,7`, and `7,7`.

For now, **only consider horizontal and vertical lines**: lines where either `x1 = x2` or `y1 = y2`.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```text
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is `0,0` and the bottom right corner is `9,9`. Each position is shown as **the number of lines which cover that point** or `.` if no line covers that point. The top-left pair of `1s`, for example, comes from `2,2 -> 2,1`; the very bottom row is formed by the overlapping lines `0,9 -> 5,9` and `0,9 -> 2,9`.

To avoid the most dangerous areas, you need to determine **the number of points where at least two lines overlap**. In the above example, this is anywhere in the diagram with a `2` or larger - a total of `5` points.

Consider only horizontal and vertical lines. **At how many points do at least two lines overlap?**

## Load Vent Vectors

In [1]:
import numpy as np
import re

# with open("example", "r") as file:
with open("input", "r") as file:
    vent_endpoints = file.read().splitlines()


## Helper Functions

In [2]:
def new_vector(x1: int, y1: int, x2: int, y2: int, incl_diag: bool = False) -> list:
    if x1 != x2 and y1 == y2:
        return [(x, y1) for x in range(min(x1, x2), (max(x1, x2) + 1))]
    elif x1 == x2 and y1 != y2:
        return [(x1, y) for y in range(min(y1, y2), (max(y1, y2) + 1))]
    elif incl_diag:
        x_points = [x for x in range(min(x1, x2), max(x1, x2) + 1)]
        y_points = [y for y in range(min(y1, y2), max(y1, y2) + 1)]

        # If first point does not match either of the original x/y points,
        # reverse the y points to realign them with the x points.
        if (x_points[0], y_points[0]) not in [(x1, y1), (x2, y2)]:
            y_points.reverse()

        # Zip x/y pairs up into tuples in a list.
        return list(zip(x_points, y_points))
    else:
        return []


def expand_field(vent_field: np.array, x1: int, y1: int, x2: int, y2: int) -> np.array:

    # Expand columns if needed.
    num_rows, num_cols = vent_field.shape
    maxx = max(x1, x2)
    if maxx >= num_cols:
        new_cols = np.zeros(shape=(num_rows, maxx - num_cols + 1), dtype=int)
        vent_field = np.hstack([vent_field, new_cols])

    # Expland rows if needed.
    num_rows, num_cols = vent_field.shape
    maxy = max(y1, y2)
    if maxy >= num_rows:
        new_cols = np.zeros(shape=(maxy - num_rows + 1, num_cols), dtype=int)
        vent_field = np.vstack([vent_field, new_cols])

    return vent_field


def mark_field(field: np.array, vector: list) -> None:
    for col_x, row_y in vector:
        field[row_y][col_x] += 1


## Part 1 : Puzzle Solution

In [3]:
def danger_zone_analysis(vent_endpoints: list, incl_diag: bool = False) -> np.array:

    vent_field = np.zeros(shape=(1, 1), dtype=int)

    for endpoints in vent_endpoints:

        # Parse endpoints into a full vector.
        x1, y1, x2, y2 = [int(x) for x in re.split(r" -> |,", endpoints)]
        vector = new_vector(x1, y1, x2, y2, incl_diag=incl_diag)

        # Expand the vent field acording to new endpoints.
        vent_field = expand_field(vent_field, x1, y1, x2, y2)

        # Mark new vector on field.
        mark_field(vent_field, vector)

    return vent_field


vent_field = danger_zone_analysis(vent_endpoints)

danger_zone_count = len(vent_field[vent_field > 1])

print(f"Danger Zones: {danger_zone_count}")
# print(f"Vent Field:\n{vent_field}")


Danger Zones: 7142


## Part 2 : Puzzle Description

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider **diagonal lines**.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like `1,1 -> 3,3` covers points `1,1`, `2,2`, and `3,3`.
- An entry like `9,7 -> 7,9` covers points `9,7`, `8,8`, and `7,9`.

Considering all lines from the above example would now produce the following diagram:

```text
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine **the number of points where at least two lines overlap**. In the above example, this is still anywhere in the diagram with a `2` or larger - now a total of `12` points.

Consider all of the lines. **At how many points do at least two lines overlap?**

## Part 2 : Puzzle Solution

In [4]:
vent_field = danger_zone_analysis(vent_endpoints, incl_diag=True)

danger_zone_count = len(vent_field[vent_field > 1])

print(f"Danger Zones: {danger_zone_count}")
# print(f"Vent Field:\n{vent_field}")

Danger Zones: 20012
